In [2]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [3]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"

warnings.filterwarnings("ignore")

In [4]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [5]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [6]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [7]:
# 오늘 날짜
today = datetime.today()#.strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제, 오늘이 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)
if today.weekday() == 5:  # 토요일
    today -= timedelta(days=1)
elif today.weekday() == 6:  # 일요일
    today -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')
today = today.strftime('%Y%m%d')

#### 법정동코드

In [8]:
dist_code = pd.read_csv('C:\py_src\midoproject\data/법정동코드 전체자료.csv',encoding='cp949')

In [9]:
dist_code_list = []
for i in range(dist_code['법정동명'].str.split().apply(len).max()):
    dist_code_temp = dist_code[dist_code['법정동명'].str.split().str[i].str[-1].isin(['읍','면','동','리','가','로']).dropna()]
    dist_code_list.append([dist_code_temp['법정동명'].str.split().str[i].dropna().unique()])

emd_code_list = list(chain(*chain(*dist_code_list)))

#### 종합쇼핑몰 납품상세내역

In [10]:
all_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_all', bigquerykey_path)

In [11]:
all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '수요기관코드', '수요기관구분', '수요기관지역명','납품요구지청명']]
all_shop_df_fin = all_shop_df_fin.sort_values(['납품요구접수일자'],ascending=False).reset_index(drop=True)

In [12]:
# 특수문자, 숫자, 영어 제거 함수 (연속된 공백을 단일 공백으로 변환 포함)
def clean_text(text):
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', ' ', text)
    text = re.sub('\s+', ' ', text).strip()  # 연속된 공백을 단일 공백으로 변환
    return text

In [13]:
# 사업명 전처리
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명'].apply(clean_text)
# all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.strip()

all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.split().apply(lambda x: [kw for kw in x if len(kw) > 1]) ## 글자수 1 제외
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].apply((' ').join)

In [14]:
# 종합쇼핑몰 학교 관련 사업
shop_edu_mapping_temp1 = all_shop_df_fin[all_shop_df_fin['납품요구건명'].str.contains('학교|교육|교육청|교육지원청|교육부|공립|사립|국립')]
shop_edu_mapping_temp2 = all_shop_df_fin[all_shop_df_fin['수요기관명'].str.contains('학교|교육|교육청|교육지원청|교육부|공립|사립|국립')]
shop_edu_mapping_df = pd.concat([shop_edu_mapping_temp1,shop_edu_mapping_temp2],axis=0).drop_duplicates().reset_index(drop=True)
shop_edu_mapping_df

,납품요구접수일자,수요기관명,납품요구건명,업체명,단가,단위,수량,금액,수요기관코드,수요기관구분,수요기관지역명,납품요구지청명,납품요구건명_re
0,2024-07-10,대전광역시동부교육청 동명중학교,동명중학교 운동장 환경개선 조성 및 기타공사 인조잔디 구매,주식회사 베노,46500,㎡,1097.0,51010500,7440035,기타기관,대전광역시 중구,대전지방조달청,동명중학교 운동장 환경개선 조성 기타공사 인조잔디 구매
1,2024-07-09,대구광역시남부교육청 성지중학교,성지중학교 운동장 인조잔디( t55mm) 구입,주식회사 플랜에이,63200,㎡,2727.0,172346400,7270193,교육기관,대구광역시 달서구,대구지방조달청,성지중학교 운동장 인조잔디 구입
2,2024-07-09,보건복지부 국립소록도병원,국립소록도병원 테니스장 보수공사 관급자재(인조잔디),주식회사 플랜에이,38000,㎡,663.0,25194000,1352331,국가기관,전라남도 고흥군,광주지방조달청,국립소록도병원 테니스장 보수공사 관급자재 인조잔디
3,2024-07-05,부산광역시해운대교육청 재송중학교,재송중학교 인조잔디 및 배수판 구입,주식회사 이원,63800,㎡,2161.0,137871800,7210042,교육기관,부산광역시 해운대구,부산지방조달청,재송중학교 인조잔디 배수판 구입
4,2024-07-05,경기도교육청 운산고등학교,운산고등학교 인조잔디 구매,주식회사 이원,63800,㎡,2985.0,190443000,7530982,교육기관,경기도 광명시,인천지방조달청,운산고등학교 인조잔디 구매
...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,2024-01-09,서울특별시강동교육청 서울가동초등학교,실내운동장 인조잔디 구입,주식회사 필드글로벌,63000,㎡,495.0,31185000,7070052,교육기관,서울특별시 송파구,서울지방조달청,실내운동장 인조잔디 구입
247,2024-01-08,제주특별자치도교육청 제주시교육지원청 동광초등학교,동광초 인조잔디 및 울타리 조성공사 관급자재 구입(인조잔디 외 1종),주식회사 필드글로벌,63000,㎡,2680.0,168840000,9296060,교육기관,제주특별자치도 제주시,제주지방조달청,동광초 인조잔디 울타리 조성공사 관급자재 구입 인조잔디
248,2024-01-04,한국과학기술원부설한국과학영재학교,풋살장 조성 및 기타공사 관급자재 구매,주식회사 이원,38000,㎡,747.0,28386000,B552516,기타공공기관,부산광역시 부산진구,부산지방조달청,풋살장 조성 기타공사 관급자재 구매
249,2024-01-03,경기도파주교육청 파주중학교,파주중 인조잔디 운동장 조성사업 관급자재 구입,주식회사 지에스케이,61500,㎡,2540.0,156210000,7680017,교육기관,경기도 파주시,서울지방조달청,파주중 인조잔디 운동장 조성사업 관급자재 구입


In [15]:
# 교육청, 교육지원청, 학교명 필터링
shop_edu_mapping_df['수요기관명_re'] = shop_edu_mapping_df['수요기관명'].str.replace('교육청','').str.replace('교육지원청','')
shop_edu_kwd = set(chain(*shop_edu_mapping_df['수요기관명_re'].str.split()))

In [16]:
# 학교 관련 키워드 데이터셋
edu_mapping_keywd_all = list(chain(*shop_edu_mapping_df['납품요구건명_re'].str.split(' '))) ## 전체 split 키워드
edu_mapping_keywd_df = pd.DataFrame(Counter(edu_mapping_keywd_all).items(),columns=['키워드','빈도수'])
edu_mapping_keywd_df = edu_mapping_keywd_df[edu_mapping_keywd_df['키워드'].apply(len)!=1].reset_index(drop=True)
edu_mapping_keywd_df.sort_values('빈도수',ascending=False)

,키워드,빈도수
5,인조잔디,258
12,관급자재,139
8,구입,106
1,운동장,97
6,구매,76
...,...,...
167,목포영산초등학교,1
166,고창북중학교,1
165,연초중,1
164,자재,1


#### 교육청 예산서

In [17]:
# 빅쿼리에서 불러오기
edu_budget_df = get_dataframe_from_bigquery('edu','edu_budget',bigquerykey_path)
edu_budget_df.sort_values('시도').reset_index(drop=True)

,시도,시군구,구분,과업명,금액,면적,예산집행
0,강원특별자치도,화천교육지원청,공립,상서중학교 운동장 보수,"89,533,000","4,664",학교
1,강원특별자치도,인제교육지원청,공립,상남중학교 개방형 운동장 조성,"725,302,000","6,507",교육청
2,강원특별자치도,강원특별자치도교육청,공립,강릉제일고등학교 축구장 인조잔디 교체,"1,020,560,000","8,576",교육청
3,강원특별자치도,강릉교육지원청,공립,관동중학교 인조잔디 운동장 조성,"806,400,000","4,082",교육청
4,강원특별자치도,강원특별자치도교육청,사립,동해광희고등학교 인조잔디운동장 보수,"51,700,000",None,None
...,...,...,...,...,...,...,...
189,충청남도,보령교육지원청,공립,성주초등학교 운동장 교체,"70,980,000",None,None
190,충청남도,당진교육지원청,공립,면천중학교 운동장 교체,"297,561,000",None,None
191,충청북도,충청북도교육청,공립,충북산업과학고등학교 다목적구장 조성,"769,799,000","5,540",교육청
192,충청북도,청주교육지원청,공립,서경중학교 운동장 보수 및 바닥교체,"602,871,000","3,500",교육청


In [18]:
# 전처리
edu_budget_df['과업명_re'] = edu_budget_df['과업명'].apply(clean_text)
# edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.strip()

In [19]:
# 지역명, 교육청명 종합쇼핑몰 형식과 통일
# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
edu_budget_df['시도_re'] = edu_budget_df['시도'].replace(replacements)
edu_budget_df['시도_re'] = edu_budget_df['시도_re'].str[:2]

edu_budget_df['시군구_re'] = np.where(edu_budget_df['시군구'].str.contains('교육지원청'),
                                   edu_budget_df['시도'] + edu_budget_df['시군구'].str.replace('교육지원청',''), edu_budget_df['시군구']) 

In [20]:
# 교육청, 교육지원청, 학교명 기준 추출
filtered_edu_budget_df = pd.concat([edu_budget_df[edu_budget_df['시군구_re'].isin(shop_edu_kwd)],
                                    edu_budget_df[edu_budget_df['과업명'].str.contains('|'.join(shop_edu_kwd))],
                                    edu_budget_df[edu_budget_df['과업명_re'].str.contains('|'.join(shop_edu_kwd))]],axis=0).drop_duplicates().reset_index(drop=True)

In [21]:
filtered_edu_budget_df

,시도,시군구,구분,과업명,금액,면적,예산집행,과업명_re,시도_re,시군구_re
0,전라남도,나주교육지원청,None,"운동장 조성 (초1, 중1)","1,126,502,000",None,교육청,운동장 조성 초 중,전남,전라남도나주
1,전라남도,나주교육지원청,None,운동장개보수 (다시초),"373,750,000","1,023",교육청,운동장개보수 다시초,전남,전라남도나주
2,강원특별자치도,춘천교육지원청,공립,부안초등학교 인조잔디 운동장 및 우레탄조성,"322,110,000",None,None,부안초등학교 인조잔디 운동장 및 우레탄조성,강원,강원특별자치도춘천
3,강원특별자치도,춘천교육지원청,공립,봉의초등학교 테니스장교체,"613,290,000",None,None,봉의초등학교 테니스장교체,강원,강원특별자치도춘천
4,강원특별자치도,춘천교육지원청,공립,춘천초등학교 다목적구장조성,"394,300,000",None,None,춘천초등학교 다목적구장조성,강원,강원특별자치도춘천
...,...,...,...,...,...,...,...,...,...,...
61,인천광역시,인천광역시교육청,공립,삼산고등학교 노후체육시설 개선,"176,000,000",None,학교,삼산고등학교 노후체육시설 개선,인천,인천광역시교육청
62,대구광역시,대구광역시교육청,공립,대구공업고등학교 축구장 인조잔디 재조성,"638,600,000","6,779",학교,대구공업고등학교 축구장 인조잔디 재조성,대구,대구광역시교육청
63,대구광역시,대구광역시교육청,공립,경북고등학교 야구장 인조잔디 재조성,"602,243,000","11,109",교육청,경북고등학교 야구장 인조잔디 재조성,대구,대구광역시교육청
64,광주광역시,광주광역시교육청,사립,광주석산고등학교(인조잔디) (학교회계전출금),"120,000,000",None,None,광주석산고등학교 인조잔디 학교회계전출금,광주,광주광역시교육청


#### 교육청 종합쇼핑몰 전처리

In [65]:
# Initialize the dictionary for split keywords
shop_split_kwd = {}
for i in range(all_shop_df_fin['납품요구건명_re'].str.split().apply(len).max()):
    shop_split_kwd['kwd_' + str(i)] = all_shop_df_fin['납품요구건명_re'].str.split().str[i]

In [79]:
# 매핑데이터셋
mapping_merge_df = []

In [80]:
# 종합쇼핑몰 납품요구건명 split 키워드 순서대로 맵핑
# Initialize the list to store the results
prep_edu_df_today = filtered_edu_budget_df.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

# Iterate over each keyword column
for i in range(len(shop_split_kwd)):
    keyword_column = 'kwd_' + str(i)
    
    # Get unique non-null keywords
    shop_split_kwd_temp = shop_split_kwd[keyword_column].dropna().unique()

    prep_edu_df_today[keyword_column] = prep_edu_df_today['과업명_re'].apply(lambda x: ' '.join(re.findall('|'.join(shop_split_kwd_temp), x)))
    prep_edu_df_today[keyword_column] = prep_edu_df_today[keyword_column].str.strip()
    prep_edu_df_today[keyword_column] = prep_edu_df_today[keyword_column].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall('|'.join(shop_split_kwd_temp), x)))
    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin[keyword_column].str.strip()
    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin[keyword_column].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

    mapping_merge_df_temp = pd.merge(
            prep_edu_df_today[(prep_edu_df_today[keyword_column] != '') & ~(prep_edu_df_today[keyword_column].isnull())][['시도','시군구','과업명', '과업명_re','시군구_re', keyword_column]].reset_index(drop=True),
            prep_all_shop_df_fin[(prep_all_shop_df_fin[keyword_column] != '') & ~(prep_all_shop_df_fin[keyword_column].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', keyword_column]].reset_index(drop=True),
            how='left',
            on=keyword_column
            ).drop_duplicates().reset_index(drop=True)
    
    # Append the temporary DataFrame to the results list
    mapping_merge_df.append(mapping_merge_df_temp)

In [81]:
# 종합쇼핑몰 지역 키워드 맵핑 - 세부사업명 지역 및 해당지역 추출
# Initialize the columns for district keywords
prep_edu_df_today = filtered_edu_budget_df.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

sig_pat = '|'.join(dist_nm)

prep_edu_df_today['kwd_sig'] = prep_edu_df_today['과업명_re'].apply(lambda x: ' '.join(re.findall(sig_pat, x)))
prep_edu_df_today['kwd_sig'] = prep_edu_df_today['kwd_sig'].str.strip()
prep_edu_df_today['kwd_sig'] = prep_edu_df_today['kwd_sig'].str.split()
prep_edu_df_today = prep_edu_df_today.explode('kwd_sig')

prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(sig_pat, x)))
prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['kwd_sig'].str.strip()
prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['kwd_sig'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_sig')

mapping_merge_df_sig = pd.merge(
    prep_edu_df_today[(prep_edu_df_today['kwd_sig'] != '') & ~(prep_edu_df_today['kwd_sig'].isnull())][['시도','시군구','과업명', '과업명_re','시군구_re', 'kwd_sig']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_sig'] != '') & ~(prep_all_shop_df_fin['kwd_sig'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_sig']].reset_index(drop=True),
    how='left',
    on='kwd_sig'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_sig)

In [82]:
# 종합쇼핑몰 지역 키워드 맵핑 - 세부사업명 지역 및 해당지역 읍면동 추출
# Initialize the columns for district keywords
prep_edu_df_today = filtered_edu_budget_df.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

emd_pat = '|'.join(set(emd_code_list))

prep_edu_df_today['kwd_emd'] = prep_edu_df_today['과업명_re'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
prep_edu_df_today['kwd_emd'] = prep_edu_df_today['kwd_emd'].str.strip()
prep_edu_df_today['kwd_emd'] = prep_edu_df_today['kwd_emd'].str.split()
prep_edu_df_today = prep_edu_df_today.explode('kwd_emd')

prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['kwd_emd'].str.strip()
prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['kwd_emd'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_emd')

mapping_merge_df_emd = pd.merge(
    prep_edu_df_today[(prep_edu_df_today['kwd_emd'] != '') & ~(prep_edu_df_today['kwd_emd'].isnull())][['시도','시군구','과업명', '과업명_re','시군구_re', 'kwd_emd']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_emd'] != '') & ~(prep_all_shop_df_fin['kwd_emd'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_emd']].reset_index(drop=True),
    how='left',
    on='kwd_emd'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_emd)

In [86]:
# 필요키워드 앞부분 추출
need_edu_kwd = '학교|초등학교|중학교|고등학교|대학교|교육|교육청|교육지원청|교육부|공립|사립|국립'
need_kwd_split = list(set(chain(*all_shop_df_fin['납품요구건명_re'].apply(lambda x: re.split(need_edu_kwd,x)).apply(''.join).str.split())))
need_kwd_split_list = list(pd.Series(need_kwd_split)[pd.Series(need_kwd_split).apply(len) >= 2])

In [88]:
# 종합쇼핑몰 지역 키워드 맵핑 - 필요키워드 앞부분 매핑
# Initialize the columns for district keywords
prep_edu_df_today = filtered_edu_budget_df.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

kwd_split_pat = '|'.join(set(need_kwd_split_list))

prep_edu_df_today['kwd_need_split'] = prep_edu_df_today['과업명_re'].apply(lambda x: ' '.join(re.findall(kwd_split_pat, x)))
prep_edu_df_today['kwd_need_split'] = prep_edu_df_today['kwd_need_split'].str.strip()
prep_edu_df_today['kwd_need_split'] = prep_edu_df_today['kwd_need_split'].str.split()
prep_edu_df_today = prep_edu_df_today.explode('kwd_need_split')

prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(kwd_split_pat, x)))
prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['kwd_need_split'].str.strip()
prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['kwd_need_split'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_need_split')

mapping_merge_df_need_split = pd.merge(
    prep_edu_df_today[(prep_edu_df_today['kwd_need_split'] != '') & ~(prep_edu_df_today['kwd_need_split'].isnull())][['시도','시군구','과업명', '과업명_re','시군구_re', 'kwd_need_split']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_need_split'] != '') & ~(prep_all_shop_df_fin['kwd_need_split'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_need_split']].reset_index(drop=True),
    how='left',
    on='kwd_need_split'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_need_split)

In [90]:
# 종합쇼핑몰 지역 키워드 맵핑 - 필요키워드 매핑
# Initialize the columns for district keywords
prep_edu_df_today = filtered_edu_budget_df.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

prep_edu_df_today['kwd_need'] = prep_edu_df_today['과업명_re'].apply(lambda x: ' '.join(re.findall(need_edu_kwd, x)))
prep_edu_df_today['kwd_need'] = prep_edu_df_today['kwd_need'].str.strip()
prep_edu_df_today['kwd_need'] = prep_edu_df_today['kwd_need'].str.split()
prep_edu_df_today = prep_edu_df_today.explode('kwd_need')

prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(need_edu_kwd, x)))
prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['kwd_need'].str.strip()
prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['kwd_need'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_need')

mapping_merge_df_need = pd.merge(
    prep_edu_df_today[(prep_edu_df_today['kwd_need'] != '') & ~(prep_edu_df_today['kwd_need'].isnull())][['시도','시군구','과업명', '과업명_re','시군구_re', 'kwd_need']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_need'] != '') & ~(prep_all_shop_df_fin['kwd_need'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_need']].reset_index(drop=True),
    how='left',
    on='kwd_need'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_need)

In [91]:
# 키워드별 데이터셋 병합
mapping_merge_df_fin = pd.concat(mapping_merge_df, axis=0).drop_duplicates()
mapping_merge_df_fin = mapping_merge_df_fin[~mapping_merge_df_fin['납품요구건명'].isnull()].reset_index(drop=True)

In [93]:
# 모든 키워드 합
kwd_columns = ['kwd_0','kwd_1','kwd_2','kwd_3','kwd_4','kwd_5','kwd_6','kwd_7','kwd_8','kwd_sig','kwd_emd','kwd_need_split','kwd_need']
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin[kwd_columns].apply(lambda x : (' ').join(x.dropna()),axis=1)
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin['all_kwd'].str.strip()
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin['all_kwd'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_fin.drop(columns=kwd_columns, inplace=True)

In [95]:
# 지역명 추출1
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명'].str.split().str[0]

# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명_split1'].replace(replacements)
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명_split1'].str[:2]

In [96]:
# 지역명 추출2
mapping_merge_df_fin['수요기관지역명_split2'] = mapping_merge_df_fin['수요기관지역명'].str.split().str[1]
dist_sig_pat = '|'.join(mapping_merge_df_fin['수요기관지역명_split2'].dropna().unique())

mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명'].apply(lambda x: ' '.join(re.findall(dist_sig_pat, x)))
mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명_re'].str.strip()
mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명_re'].str.split()
mapping_merge_df_fin = mapping_merge_df_fin.explode('자치단체명_re')

KeyError: '자치단체명'

In [40]:
# 제외 지역 제거
filtered_mapping_merge_df_fin = mapping_merge_df_fin[mapping_merge_df_fin['자치단체명_re']!=''].drop_duplicates().reset_index(drop=True)

In [41]:
# 지역 일치 확인1
mapping_merge_df_fin_temp1 = filtered_mapping_merge_df_fin[filtered_mapping_merge_df_fin['지역명']==filtered_mapping_merge_df_fin['수요기관지역명_split1']].drop_duplicates().reset_index(drop=True)
mapping_merge_df_fin_temp1 = mapping_merge_df_fin_temp1[mapping_merge_df_fin_temp1['자치단체명'].str.contains('|'.join(mapping_merge_df_fin_temp1['수요기관지역명_split2'].dropna().unique()))].drop_duplicates().reset_index(drop=True)
mapping_merge_df_fin_temp1 = mapping_merge_df_fin_temp1[mapping_merge_df_fin_temp1['자치단체명_re']==mapping_merge_df_fin_temp1['수요기관지역명_split2']].drop_duplicates().reset_index(drop=True)

# 본청 포함 확인
mapping_merge_df_fin_temp2 = filtered_mapping_merge_df_fin[filtered_mapping_merge_df_fin['자치단체명'].str.contains('본청')].drop_duplicates().reset_index(drop=True)

# 데이터 병합
mapping_merge_df_final = pd.concat([mapping_merge_df_fin_temp1,mapping_merge_df_fin_temp2], axis=0).drop_duplicates().reset_index(drop=True)

In [42]:
# 그룹화할 컬럼 리스트
group_cols = ['지역명', '자치단체명', '세부사업코드', '세부사업명', '세부사업명_re', '수요기관명', '수요기관지역명', '납품요구건명', '납품요구건명_re','수요기관지역명_split1', '자치단체명_re', '수요기관지역명_split2']

# 그룹화 및 all_kwd 컬럼 조인
mapping_merge_df_final_grp = mapping_merge_df_final.groupby(group_cols).agg({'all_kwd': ' '.join}).reset_index()
mapping_merge_df_final_grp['all_kwd'] = mapping_merge_df_final_grp['all_kwd'].str.strip()
mapping_merge_df_final_grp['all_kwd'] = mapping_merge_df_final_grp['all_kwd'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [43]:
# 필요키워드 기준
mapping_merge_df_final_grp['세부사업명_키워드1'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))
mapping_merge_df_final_grp['납품요구건명_키워드1'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))

# 사업명 지역 추출
mapping_merge_df_final_grp['세부사업명_지역'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
mapping_merge_df_final_grp['납품요구건명_지역'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))

# 필요키워드 패턴 기준
need_kwd_pat = '\w+' + ('|\w+').join(need_kwd1.split('|'))

mapping_merge_df_final_grp['세부사업명_키워드2'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(need_kwd_pat, x)))
mapping_merge_df_final_grp['납품요구건명_키워드2'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(need_kwd_pat, x)))

In [44]:
def check_inclusion1(row):
    set1 = set(row['세부사업명_키워드1'])
    set2 = set(row['납품요구건명_키워드1'])
    return bool(set1 & set2)

In [45]:
# 필요키워드 모두 없는것 제외
mapping_merge_df_final_grp_temp1 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_키워드1']!='')&(mapping_merge_df_final_grp['납품요구건명_키워드1']!='')].reset_index(drop=True)

# 필요키워드 포함여부 확인
mapping_merge_df_final_grp_temp1['세부사업명_키워드1'] = mapping_merge_df_final_grp_temp1['세부사업명_키워드1'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'] = mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_temp1 = mapping_merge_df_final_grp_temp1[mapping_merge_df_final_grp_temp1.apply(check_inclusion1,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp1['세부사업명_키워드1'] = mapping_merge_df_final_grp_temp1['세부사업명_키워드1'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'] = mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [46]:
def check_inclusion_dist(row):
    set1 = set(row['세부사업명_지역'])
    set2 = set(row['납품요구건명_지역'])
    return bool(set1 & set2)

In [47]:
# 지역 없는것 제외
mapping_merge_df_final_grp_temp2 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_지역']!='')&(mapping_merge_df_final_grp['납품요구건명_지역']!='')].reset_index(drop=True)

# 사업명 지역 포함여부 확인
mapping_merge_df_final_grp_temp2['세부사업명_지역'] = mapping_merge_df_final_grp_temp2['세부사업명_지역'].str.split()
mapping_merge_df_final_grp_temp2['납품요구건명_지역'] = mapping_merge_df_final_grp_temp2['납품요구건명_지역'].str.split()

mapping_merge_df_final_grp_temp2 = mapping_merge_df_final_grp_temp2[mapping_merge_df_final_grp_temp2.apply(check_inclusion_dist,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp2['세부사업명_지역'] = mapping_merge_df_final_grp_temp2['세부사업명_지역'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip() ## 키워드별 정리
mapping_merge_df_final_grp_temp2['납품요구건명_지역'] = mapping_merge_df_final_grp_temp2['납품요구건명_지역'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip() ## 키워드별 정리

In [48]:
def check_inclusion2(row):
    set1 = set(row['세부사업명_키워드2'])
    set2 = set(row['납품요구건명_키워드2'])
    return bool(set1 & set2)

In [49]:
# 필요키워드패턴 모두 없는것 제외
mapping_merge_df_final_grp_temp3 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_키워드2']!='')&(mapping_merge_df_final_grp['납품요구건명_키워드2']!='')].reset_index(drop=True)

# 필요키워드 패턴 포함여부 확인
mapping_merge_df_final_grp_temp3['세부사업명_키워드2'] = mapping_merge_df_final_grp_temp3['세부사업명_키워드2'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'] = mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_temp3 = mapping_merge_df_final_grp_temp3[mapping_merge_df_final_grp_temp3.apply(check_inclusion2,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp3['세부사업명_키워드2'] = mapping_merge_df_final_grp_temp3['세부사업명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'] = mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [50]:
# 필터링 데이터셋 병합
mapping_merge_df_final_grp_fin = pd.concat([mapping_merge_df_final_grp_temp1,mapping_merge_df_final_grp_temp2,mapping_merge_df_final_grp_temp3]).drop_duplicates().reset_index(drop=True)
mapping_merge_df_final_grp_fin

,지역명,자치단체명,세부사업코드,세부사업명,세부사업명_re,수요기관명,수요기관지역명,납품요구건명,납품요구건명_re,수요기관지역명_split1,자치단체명_re,수요기관지역명_split2,all_kwd,세부사업명_키워드1,납품요구건명_키워드1,세부사업명_지역,납품요구건명_지역,세부사업명_키워드2,납품요구건명_키워드2
0,강원,강원동해시,4211000202430032,발한게이트볼장 조성사업(전환사업),발한게이트볼장 조성사업 전환사업,강원특별자치도 동해시,강원도 동해시,샘실8단지게이트볼장 인조잔디 구입 및 설치,샘실 단지게이트볼장 인조잔디 구입 설치,강원,동해시,동해시,게이트볼장,게이트볼장,게이트볼장,,,발한게이트볼장,샘실8단지게이트볼장
1,강원,강원삼척시,42400002023302CB,도계 전천후 테니스장 조성사업(도)(전환사업),도계 전천후 테니스장 조성사업 전환사업,강원특별자치도 삼척시,강원도 삼척시,도계늑구테니스장정비공사관급자재[인조잔디],도계늑구테니스장정비공사관급자재 인조잔디,강원,삼척시,삼척시,테니스 테니스장,테니스장,관급 테니스장,,,,도계늑구테니스장 정비공사관급
2,강원,강원속초시,423000020203003D,장기미집행 도시공원 조성,장기미집행 도시공원 조성,강원특별자치도 속초시,강원도 속초시,(관급자재)설악동 화채마을 수변공원 조성사업 관급 인조잔디 구입,관급자재 설악동 화채마을 수변공원 조성사업 관급 인조잔디 구입,강원,속초시,속초시,공원 조성,공원,공원 관급,,설악동,도시공원,수변공원
3,강원,강원속초시,423000020203003D,장기미집행 도시공원 조성,장기미집행 도시공원 조성,강원특별자치도 속초시,강원도 속초시,(관급자재)설악동 화채마을 수변공원 조성사업 인조잔디 추가 구입,관급자재 설악동 화채마을 수변공원 조성사업 인조잔디 추가 구입,강원,속초시,속초시,공원 조성,공원,공원 관급,,설악동,도시공원,수변공원
4,강원,강원속초시,4231000202430050,속초 교동게이트볼장 환경개선사업,속초 교동게이트볼장 환경개선사업,강원특별자치도 속초시,강원도 속초시,노학동 게이트볼장 기능개선공사(건축) 관급 인조잔디,노학동 게이트볼장 기능개선공사 건축 관급 인조잔디,강원,속초시,속초시,게이트볼장,게이트볼장,게이트볼장 관급,교동,노학동,교동게이트볼장,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2246,충북,충북음성군,4470000202430032,감곡생활체육공원 정비공사,감곡생활체육공원 정비공사,충청북도 음성군 시설관리사업소,충청북도 음성군,감곡생활체육공원 전천후게이트볼장 인조잔디 설치 관급자재 구입,감곡생활체육공원 전천후게이트볼장 인조잔디 설치 관급자재 구입,충북,음성군,음성군,감곡생활 체육 체육공원,체육공원,체육공원 게이트볼장 관급,,,감곡생활체육공원,감곡생활체육공원 전천후게이트볼장
2247,충북,충북증평군,5570000202030037,증평생활야구장 조성,증평생활야구장 조성,충청북도 증평군,충청북도 증평군,증평생활야구장 시설보안공사 관급자재 구입,증평생활야구장 시설보안공사 관급자재 구입,충북,증평군,증평군,야구장 증평 증평생활,야구장,야구장 관급,,,증평생활야구장,증평생활야구장
2248,충북,충북증평군,5570000202030037,증평생활야구장 조성,증평생활야구장 조성,충청북도 증평군,충청북도 증평군,증평생활야구장 시설보완공사 관급자재 구입,증평생활야구장 시설보완공사 관급자재 구입,충북,증평군,증평군,야구장 증평 증평생활,야구장,야구장 관급,,,증평생활야구장,증평생활야구장
2249,충북,충북청주시,57100002017303BA,내수야구장 조성사업,내수야구장 조성사업,충청북도 청주시,충청북도 청주시 상당구,"내수야구장 조성사업(조경, 토목) 관급자재(인조잔디)",내수야구장 조성사업 조경 토목 관급자재 인조잔디,충북,청주시,청주시,내수 사업 야구장 조성,야구장,야구장 관급,,,내수야구장,내수야구장


In [51]:
# 모든 키워드 합 중복 제거
kwd_columns = ['all_kwd','세부사업명_키워드1', '납품요구건명_키워드1', '세부사업명_지역', '납품요구건명_지역', '세부사업명_키워드2', '납품요구건명_키워드2']
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin[kwd_columns].apply(lambda x : (' ').join(x.dropna()),axis=1)
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin['all_kwd_fin'].str.strip()
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin['all_kwd_fin'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
# mapping_merge_df_final_grp_fin.drop(columns=kwd_columns, inplace
# =True)

mapping_merge_df_final_grp_fin = mapping_merge_df_final_grp_fin.drop_duplicates(['지역명', '자치단체명', '세부사업코드', '세부사업명', '세부사업명_re', '수요기관명', '수요기관지역명',
                                                                                 '납품요구건명', '납품요구건명_re', '수요기관지역명_split1', '자치단체명_re', '수요기관지역명_split2', 'all_kwd_fin']).reset_index(drop=True)

In [52]:
# 최종필터링1 - 지역 일치
mapping_merge_df_final_grp_fin_temp1 = mapping_merge_df_final_grp_fin[(mapping_merge_df_final_grp_fin['세부사업명_지역']!='') & (mapping_merge_df_final_grp_fin['납품요구건명_지역']!='')]
mapping_merge_df_final_grp_fin_temp1 = mapping_merge_df_final_grp_fin_temp1[mapping_merge_df_final_grp_fin_temp1['세부사업명_지역']==mapping_merge_df_final_grp_fin_temp1['납품요구건명_지역']]

In [53]:
# 최종필터링2 - 필수키워드 패턴 포함 & 지역 불일치 제거
mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin[(mapping_merge_df_final_grp_fin['세부사업명_키워드2']!='')&(mapping_merge_df_final_grp_fin['납품요구건명_키워드2']!='')].reset_index(drop=True)

# 필요키워드 패턴 포함여부 확인
mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin_temp2[mapping_merge_df_final_grp_fin_temp2.apply(check_inclusion2,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

# 지역 불일치 제거
mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin_temp2[mapping_merge_df_final_grp_fin_temp2['세부사업명_지역']==mapping_merge_df_final_grp_fin_temp2['납품요구건명_지역']]

In [54]:
# 최종 필터링 데이터셋
mapping_merge_df_final_grp_final = pd.concat([mapping_merge_df_final_grp_fin_temp1,mapping_merge_df_final_grp_fin_temp2]).drop_duplicates().reset_index(drop=True)
mapping_merge_df_final_grp_final.drop(columns = 'all_kwd', inplace=True)

In [57]:
# mapping_merge_df_final_grp_final.to_csv('C:\py_src\midoproject\data/filtered_mapping_df.csv',encoding='utf-8-sig',index=False)

In [58]:
# 종합쇼핑몰 기준 필터링
bid_cd1 = list(budget_df_today[budget_df_today['세부사업코드'].isin(mapping_merge_df_final_grp_final['세부사업코드'])]['세부사업코드'])
bid_cd2 = list(budget_df_today[budget_df_today['세부사업명'].isin(mapping_merge_df_final_grp_final['세부사업명'])]['세부사업코드'])

In [59]:
budget_df_today_fin = budget_df_today[~budget_df_today['세부사업코드'].isin(set(bid_cd1 + bid_cd2))].drop_duplicates().reset_index(drop=True)

In [60]:
# 중요 사업 체크
budget_df_today_final = pd.concat([budget_df_today_fin[budget_df_today_fin['세부사업명'].str.contains('인조잔디')],
                                   budget_df_today_fin[~budget_df_today_fin['세부사업명'].str.contains('인조잔디')]],axis=0).reset_index(drop=True)

# budget_df_ytday_final = pd.concat([budget_df_ytday_fin[budget_df_ytday_fin['세부사업명'].str.contains('인조잔디')],
#                                    budget_df_ytday_fin[~budget_df_ytday_fin['세부사업명'].str.contains('인조잔디')]],axis=0).reset_index(drop=True)

In [27]:
# 최종데이터셋
budget_df_today_final = budget_df_today_final[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)
budget_df_delete = budget_df_delete[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)
budget_df_new = budget_df_new[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)

In [29]:
# 빅쿼리에 적재
save_dataframe_to_bigquery(budget_df_today_final,'budget','budget_df_listup',bigquerykey_path)
save_dataframe_to_bigquery(budget_df_delete,'budget','budget_df_delete',bigquerykey_path)
save_dataframe_to_bigquery(budget_df_new,'budget','budget_df_new',bigquerykey_path)

Data inserted into table budget_df_listup successfully.
Data inserted into table budget_df_delete successfully.
Data inserted into table budget_df_new successfully.
